# Object Detection YOLO - Minimal Version (For Memorization)

YOLO pretrained model for object detection


In [ ]:
# Imports
import numpy as np
import cv2
import matplotlib.pyplot as plt
import urllib.request
import os

# Download YOLO files
yolo_config_url = "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg"
yolo_config_file = "yolov3.cfg"
yolo_weights_file = "yolov3.weights"
coco_names_url = "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"
coco_names_file = "coco.names"

def download_file(url, filename):
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

download_file(yolo_config_url, yolo_config_file)
download_file(coco_names_url, coco_names_file)


In [ ]:
# Load class names and YOLO model
with open(coco_names_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

if os.path.exists(yolo_weights_file):
    net = cv2.dnn.readNetFromDarknet(yolo_config_file, yolo_weights_file)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    print(f"YOLO loaded: {len(classes)} classes")
else:
    net = None
    output_layers = []
    print("Weights file not found!")


In [ ]:
# Load or create image
# Option 1: Load from file
# image = cv2.imread('image.jpg')
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Option 2: Create sample
from PIL import Image, ImageDraw
img = Image.new('RGB', (640, 480), 'white')
draw = ImageDraw.Draw(img)
draw.rectangle([100, 100, 300, 250], fill='blue')
image = np.array(img)
print(f"Image shape: {image.shape}")


In [ ]:
# Preprocess image
def preprocess_image(image):
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    blob = cv2.dnn.blobFromImage(image_bgr, 1/255.0, (416, 416), (0, 0, 0), swapRB=False, crop=False)
    return blob, image_bgr

blob, image_bgr = preprocess_image(image)
print(f"Blob shape: {blob.shape}")


In [ ]:
# Run detection
def detect_objects(net, blob, output_layers, conf_thresh=0.5, nms_thresh=0.4):
    if net is None:
        return [], [], []
    
    net.setInput(blob)
    outputs = net.forward(output_layers)
    
    boxes, confidences, class_ids = [], [], []
    height, width = image_bgr.shape[:2]
    
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > conf_thresh:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_thresh, nms_thresh)
    if len(indices) > 0:
        indices = indices.flatten()
        boxes = [boxes[i] for i in indices]
        confidences = [confidences[i] for i in indices]
        class_ids = [class_ids[i] for i in indices]
    
    return boxes, confidences, class_ids

boxes, confidences, class_ids = detect_objects(net, blob, output_layers)
print(f"Detections: {len(boxes)}")


In [ ]:
# Draw detections
def draw_detections(image, boxes, confidences, class_ids, classes):
    image_copy = image.copy()
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    
    for i in range(len(boxes)):
        x, y, w, h = boxes[i]
        class_id = class_ids[i]
        confidence = confidences[i]
        class_name = classes[class_id]
        color = colors[class_id].tolist()
        
        cv2.rectangle(image_copy, (x, y), (x + w, y + h), color, 2)
        label = f"{class_name}: {confidence:.2%}"
        (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(image_copy, (x, y - th - 5), (x + tw, y), color, -1)
        cv2.putText(image_copy, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    return image_copy

if len(boxes) > 0:
    result = draw_detections(image_bgr, boxes, confidences, class_ids, classes)
    result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title('Original')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(result_rgb)
    plt.title(f'Detections: {len(boxes)}')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    plt.imshow(image)
    plt.title('No detections')
    plt.axis('off')
    plt.show()


In [ ]:
# Alternative: YOLOv5 (simpler)
try:
    from ultralytics import YOLO
    model = YOLO('yolov5n.pt')
    results = model(image)
    annotated = results[0].plot()
    plt.imshow(annotated)
    plt.axis('off')
    plt.show()
    print(f"YOLOv5 detected: {len(results[0].boxes)} objects")
except:
    print("YOLOv5 not available, using OpenCV DNN")
